In [49]:
import requests
from bs4 import BeautifulSoup

 
url = 'http://phonondb.mtl.kyoto-u.ac.jp/ph20180417/index.html'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
materialsGathered = []
namesOfMaterial = {"NotAMaterial":"mat"}

html = requests.get(url)
soup = BeautifulSoup(html.text, features="html.parser")
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

for element in chunks:
    if("/" in element):
        temp = element.split("/")
        namesOfMaterial[element.split("/")[0].split(" ")[2]] = temp[1][1:len(temp[1])-1]

In [50]:
print(namesOfMaterial["992141"])

Cs2IBrCl6


In [51]:
for link in soup.find_all('a'):
    if("mp" in link.get('href')):
        urls.append(link.get('href'))
        materialsGathered.append(link.get('href').rsplit('/', 1)[-1].rsplit('.html', 1)[0])
print(urls[0])
print(materialsGathered[0])
print(len(materialsGathered))

d000/mp-25.html
mp-25
10034


In [52]:
print(len(materialsGathered))

10034


In [5]:
exampleImage = 'phononDBImages\mp-25-band.jpg'
print(exampleImage)
from PIL import Image

im = Image.open(exampleImage).convert("RGB")

pixels = {i for i in im.getdata()}

phononDBImages\mp-25-band.jpg


In [6]:
print((196,196,255) in pixels)

True


In [2]:
import numpy as np
from PIL import Image 
import cv2
import imageio
import matplotlib.pyplot as plt
import os
from numba import jit

@jit(nopython=True)
def isStable(pixels):
    purpleResult = -1
    redResult = -1
    for y in range(len(pixels)-1, 0, -1):
        for x in range(len(pixels[0])-1, 0, -1):
            if(pixels[y][x][0] == 255 and pixels[y][x][1] < 140 and redResult == -1):
                redResult = y
            if(pixels[y][x][2] > 250 and 220 > pixels[y][x][1] == pixels[y][x][0] > 190):
                purpleResult = y
            if(redResult != -1 and purpleResult != -1):
                break
        if(redResult != -1 and purpleResult != -1):
                break
    
    if(purpleResult == False or redResult == False):
        print("Cannt read: ", exampleImage)
    else:
        if(redResult - purpleResult > 3):
            return False
        else:
            return True

In [4]:
import pandas as pd
import os
results = {"mp-DOESNOTEXIST": 0}
filelist = [file for file in os.listdir("phononDBImages") if file.endswith('.jpg')]
filelist = pd.read_csv('filelist.csv',header = None)
newFilelist = []
for entry in range(len(filelist)):
    thing = filelist.iloc[entry].values[0]
    newFilelist.append(thing)
entries = pd.read_csv('entries.csv',header=None)
classification = []
materialsProcessed = []
for fileIdx in range(len(filelist)):
    files = newFilelist[fileIdx]
    exampleImage = files[:len(files)-9]
    image = Image.open("phononDBImages\\" + exampleImage + "-band.jpg").convert('RGB')
    pixels = np.asarray(image)
    results[exampleImage] = isStable(pixels)
    materialsProcessed.append(exampleImage)
    if(fileIdx % 100 == 0):
        print("Percent: ", 100*(fileIdx/len(filelist)))

Percent:  0.0
Percent:  1.0073536818777074
Percent:  2.0147073637554147
Percent:  3.0220610456331216
Percent:  4.029414727510829
Percent:  5.036768409388536
Percent:  6.044122091266243
Percent:  7.051475773143951
Percent:  8.058829455021659
Percent:  9.066183136899365
Percent:  10.073536818777072
Percent:  11.08089050065478
Percent:  12.088244182532486
Percent:  13.095597864410195
Percent:  14.102951546287901
Percent:  15.110305228165611
Percent:  16.117658910043318
Percent:  17.125012591921024
Percent:  18.13236627379873
Percent:  19.139719955676437
Percent:  20.147073637554143
Percent:  21.154427319431854
Percent:  22.16178100130956
Percent:  23.169134683187266
Percent:  24.176488365064973
Percent:  25.183842046942683
Percent:  26.19119572882039
Percent:  27.198549410698096
Percent:  28.205903092575802
Percent:  29.213256774453512
Percent:  30.220610456331222
Percent:  31.227964138208925
Percent:  32.235317820086635
Percent:  33.24267150196434
Percent:  34.25002518384205
Percent:  35

In [10]:
# load json module
import json

# create json object from dictionary
json = json.dumps(results)

# open file for writing, "w" 
f = open("outputClasses.json","w")

# write json object to file
f.write(json)

# close file
f.close()

In [54]:
print(materialsProcessed[:50])

['mp-1000', 'mp-10009', 'mp-10070', 'mp-10074', 'mp-10080', 'mp-10096', 'mp-10097', 'mp-10155', 'mp-10159', 'mp-10163', 'mp-10165', 'mp-10166', 'mp-10167', 'mp-10175', 'mp-10182', 'mp-10191', 'mp-10219', 'mp-1022', 'mp-10223', 'mp-10226', 'mp-10233', 'mp-10234', 'mp-10246', 'mp-10250', 'mp-10251', 'mp-10281', 'mp-10288', 'mp-10289', 'mp-1029', 'mp-10299', 'mp-10314', 'mp-10322', 'mp-10334', 'mp-10336', 'mp-10337', 'mp-10340', 'mp-10344', 'mp-10347', 'mp-10348', 'mp-10349', 'mp-10368', 'mp-10378', 'mp-10381', 'mp-1039', 'mp-10390', 'mp-10392', 'mp-10401', 'mp-10402', 'mp-10408', 'mp-10409']


In [4]:
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from ase.db import connect
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
from pymatgen.ext.matproj import MPRester

entries = entries.transpose()

import csv
index = 0
databaseLocation = "C:\\Users\\dylan\\Desktop\\generatedData\\"
print(databaseLocation)
properties = ['dynamic_stability_phonons']
cyrstalSystems = {"tetragonal": 0}

with open(str(databaseLocation + 'dataCombinedPhononFinalV2.csv'),'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    arrayOfEntries = ['material_id','composition']
    for propertyEntry in properties:
        arrayOfEntries.append(propertyEntry)
    csvwriter.writerow(arrayOfEntries)
    offset = index
    for equationTitle in materialsProcessed:
        array = []
        array.append(index)
        datapoint = entries[index].values.tolist()
        if str(datapoint[5]) not in cyrstalSystems.keys():
            cyrstalSystems[str(datapoint[5])] = len(cyrstalSystems)
        valueZ = "Z" + str(cyrstalSystems[str(datapoint[5])])
        array.append(namesOfMaterial[materialsProcessed[index-offset][3:]] + valueZ + valueA)
        if(classification[index-offset] == True):
            array.append(1)
        else:
            array.append(0)
        csvwriter.writerow(array)
        index = index + 1

C:\Users\dylan\Desktop\generatedData\


In [ ]:
print(cyrstalSystems.keys())

In [ ]:
pic = imageio.imread(exampleImage)
plt.figure(figsize = (15,15))
plt.imshow(pic)

In [338]:
print(materialsProcessed)

['mp-1000', 'mp-10009', 'mp-10070', 'mp-10074', 'mp-10080', 'mp-10096', 'mp-10097', 'mp-10103', 'mp-10155', 'mp-10159', 'mp-10163', 'mp-10165', 'mp-10166', 'mp-10167', 'mp-10175', 'mp-10182', 'mp-10191', 'mp-10219', 'mp-1022', 'mp-10223', 'mp-10226', 'mp-10233', 'mp-10234', 'mp-10246', 'mp-10250', 'mp-10251', 'mp-10281', 'mp-10288', 'mp-10289', 'mp-1029', 'mp-10299', 'mp-10314', 'mp-10322', 'mp-10334', 'mp-10336', 'mp-10337', 'mp-10340', 'mp-10344', 'mp-10347', 'mp-10348', 'mp-10349', 'mp-10368', 'mp-10378', 'mp-10381', 'mp-1039', 'mp-10390', 'mp-10392', 'mp-10401', 'mp-10402', 'mp-10408', 'mp-10409', 'mp-10410', 'mp-10417', 'mp-10418', 'mp-10429', 'mp-10444', 'mp-1045', 'mp-10461', 'mp-1047', 'mp-10474', 'mp-10476', 'mp-10485', 'mp-10486', 'mp-10487', 'mp-10488', 'mp-10489', 'mp-10492', 'mp-10495', 'mp-10497', 'mp-10499', 'mp-10509', 'mp-10510', 'mp-10517', 'mp-10520', 'mp-10533', 'mp-10534', 'mp-10547', 'mp-10550', 'mp-10552', 'mp-10555', 'mp-10556', 'mp-1056', 'mp-10572', 'mp-10577'

In [39]:
print(entries.transpose()[0].values.tolist())

[0.1, 'spglib', 'Pa-3', 205, 'm-3', 'cubic', '-P 2ac 2ab 3']
